# TensorFlow 2.0

In [4]:
import os
from glob import glob
from datetime import datetime

import numpy as np

import tensorflow as tf
from tensorflow.keras import layers

from tensorflow.keras import datasets 

import matplotlib.pyplot as plt

%load_ext tensorboard
%matplotlib inline

## Hyperparameter Tunning

In [5]:
num_epochs = 5
batch_size = 32

learning_rate = 0.001

dropout_rate = 0.5

input_shape = (32, 32, 3)
num_classes = 10

## Build Model

In [6]:
inputs = layers.Input(input_shape)

net = layers.Conv2D(32, (3, 3), padding='SAME')(inputs)
net = layers.Activation('relu')(net)
net = layers.Conv2D(32, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.MaxPooling2D(pool_size=(2, 2))(net)
net = layers.Dropout(dropout_rate)(net)

net = layers.Conv2D(64, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.Conv2D(64, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.MaxPooling2D(pool_size=(2, 2))(net)
net = layers.Dropout(dropout_rate)(net)

net = layers.Flatten()(net)
net = layers.Dense(512)(net)
net = layers.Activation('relu')(net)
net = layers.Dropout(dropout_rate)(net)
net = layers.Dense(num_classes)(net)
net = layers.Activation('softmax')(net)

model = tf.keras.Model(inputs=inputs, outputs=net, name='Basic_CNN')

In [7]:
# Model is the full model w/o custom layers
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate),  # Optimization
              loss='sparse_categorical_crossentropy',  # Loss Function 
              metrics=['accuracy'])  # Metrics / Accuracy

# Data Preprocess

In [8]:
train_paths = glob('../dataset/cifar/train/*.png')[:100]
test_paths = glob('../dataset/cifar/test/*.png')[:100]

In [9]:
def get_class_name(path):
    return path.split('_')[-1].replace('.png', '')

In [10]:
train_labels = [get_class_name(path) for path in train_paths]
class_names = np.unique(train_labels)

In [11]:
def get_label(path):
    fname = tf.strings.split(path, '_')[-1]
    lbl_name = tf.strings.regex_replace(fname, '.png', '')
    onehot = tf.cast(lbl_name == class_names, tf.uint8)
    return tf.argmax(onehot)  # 이번에는 onehot이 아닌 label 번호로

In [12]:
def load_image_label(path):
    gfile = tf.io.read_file(path)
    image = tf.io.decode_image(gfile)
    image = tf.cast(image, tf.float32) / 255.  # rescale
    
    label = get_label(path)
    return image, label

In [13]:
def image_preprocess(image, label):
    image = tf.image.random_flip_up_down(image)
    image = tf.image.random_flip_left_right(image)
    return image, label

In [14]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

In [15]:
train_dataset = tf.data.Dataset.from_tensor_slices(train_paths)
train_dataset = train_dataset.map(load_image_label, num_parallel_calls=AUTOTUNE)
train_dataset = train_dataset.map(image_preprocess, num_parallel_calls=AUTOTUNE)
train_dataset = train_dataset.batch(batch_size)
train_dataset = train_dataset.shuffle(buffer_size=len(train_paths))
train_dataset = train_dataset.repeat()

In [16]:
test_dataset = tf.data.Dataset.from_tensor_slices(test_paths)
test_dataset = test_dataset.map(load_image_label, num_parallel_calls=AUTOTUNE)
test_dataset = test_dataset.batch(batch_size)
test_dataset = test_dataset.repeat()

## Training

http://localhost:6006

In [17]:
steps_per_epoch = len(train_paths) // batch_size
validation_steps = len(test_paths) // batch_size

history = model.fit_generator(
    train_dataset,
    steps_per_epoch=steps_per_epoch,
    validation_data=test_dataset,
    validation_steps=validation_steps,
    epochs=num_epochs
)

C:\Users\pmcsh\Anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\keras\engine\training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/5
3/3 [==============================] - 2s 496ms/step - loss: 2.3947 - accuracy: 0.1482 - val_loss: 2.2998 - val_accuracy: 0.0729
Epoch 2/5
3/3 [==============================] - 1s 209ms/step - loss: 2.3118 - accuracy: 0.1602 - val_loss: 2.3274 - val_accuracy: 0.0729
Epoch 3/5
3/3 [==============================] - 0s 179ms/step - loss: 2.3252 - accuracy: 0.1104 - val_loss: 2.3268 - val_accuracy: 0.0729
Epoch 4/5
3/3 [==============================] - 0s 156ms/step - loss: 2.3226 - accuracy: 0.0827 - val_loss: 2.3267 - val_accuracy: 0.1042
Epoch 5/5
3/3 [==============================] - 0s 209ms/step - loss: 2.2270 - accuracy: 0.2737 - val_loss: 2.3646 - val_accuracy: 0.0729


# Load Checkpoint

In [ ]:
model.load_weights('checkpoints')

# Saving Model

In [18]:
save_path = 'my_model.h5'

In [21]:
model.save(save_path, include_optimizer = True)

In [22]:
tf.keras.models.load_model('my_model.h5')

# Saving Model - 2

In [23]:
model.save_weights('model_weights.h5')

In [24]:
with open('model_architecture.json','w') as f:
    f.write(model.to_json())

In [25]:
from tensorflow.keras.models import model_from_json

In [26]:
with open('model_architecture.json', 'r') as f:
    model = model_from_json(f.read())

In [27]:
model.load_weights('model_weights.h5')

# model.h5 들여다보기

In [28]:
import h5py

In [29]:
model_file = h5py.File('my_model.h5','r+')

In [30]:
model_file.keys()

<KeysViewHDF5 ['model_weights', 'optimizer_weights']>

In [31]:
model_file['model_weights'].keys()

<KeysViewHDF5 ['activation', 'activation_1', 'activation_2', 'activation_3', 'activation_4', 'activation_5', 'conv2d', 'conv2d_1', 'conv2d_2', 'conv2d_3', 'dense', 'dense_1', 'dropout', 'dropout_1', 'dropout_2', 'flatten', 'input_1', 'max_pooling2d', 'max_pooling2d_1']>

In [32]:
model_file['model_weights']['conv2d']['conv2d'].keys()

<KeysViewHDF5 ['bias:0', 'kernel:0']>

In [34]:
weight = np.array(model_file['model_weights']['conv2d']['conv2d']['kernel:0'])

In [35]:
weight

array([[[[-0.03630106,  0.01665179,  0.03196026, -0.00125047,
           0.0248889 , -0.09144155,  0.12316304,  0.04768398,
          -0.03214347,  0.08755016,  0.10682971, -0.01939817,
           0.08086208, -0.10119227,  0.14246207, -0.11312256,
          -0.06989056,  0.08647112,  0.09962493,  0.02178361,
           0.00562367, -0.07846001,  0.09899911,  0.11335301,
           0.0134069 , -0.03890148, -0.05791107, -0.11233088,
           0.13214068, -0.07843333, -0.07753862, -0.04133137],
         [ 0.11103436,  0.00165727, -0.05803962,  0.11201317,
           0.00275071,  0.00160442,  0.10500781, -0.10609115,
           0.10911751, -0.06824061,  0.11125349,  0.05909421,
           0.04863689, -0.0667519 , -0.13360032, -0.04036625,
           0.06249348,  0.01181683, -0.07424951, -0.04570771,
          -0.03580257, -0.04388504, -0.06141111, -0.10712428,
           0.06742401,  0.05428011,  0.09716335,  0.03986028,
          -0.10059503, -0.08051846, -0.03610084,  0.05863261],
      